## 3. $H\rightarrow b\bar{b}$ via Machine Learning 

### 3.1 Importing libraries and loading 


In [ ]:
# Download the data files if we need them. If you download the repo as a ZIP, 
# this cell is not needed. If running on colab, it will automatically download
# all required data.

from pathlib import Path

data_path = Path('../data-v2/VHbb_data_2jet.csv')
train_data_path = Path('../data-v2/VHbb_data_2jet_train.csv')
val_data_path = Path('../data-v2/VHbb_data_2jet_val.csv')
test_data_path = Path('../data-v2/VHbb_data_2jet_test.csv')
class_path = Path('ucl_masterclass.py')
if not data_path.exists():
    !wget -P ../data-v2/ https://raw.githubusercontent.com/nikitapond/in2HEP/master/data-v2/VHbb_data_2jet.csv
else:
    print("Data file already found")

if not train_data_path.exists():
    !wget -P ../data-v2 https://raw.githubusercontent.com/nikitapond/in2HEP/master/data-v2/VHbb_data_2jet_train.csv
else:
    print("Train data file already found")

if not val_data_path.exists():
    !wget -P ../data-v2 https://raw.githubusercontent.com/nikitapond/in2HEP/master/data-v2/VHbb_data_2jet_val.csv
else:
    print("Validation data file already found")

if not test_data_path.exists():
    !wget -P ../data-v2 https://raw.githubusercontent.com/nikitapond/in2HEP/master/data-v2/VHbb_data_2jet_test.csv
else:
    print("Test data file already found")

if not class_path.exists():
    !wget https://raw.githubusercontent.com/nikitapond/in2HEP/master/notebooks/ucl_masterclass.py
else:
    print("Required custom classes already found")

In [ ]:
import pandas as pd
from copy import deepcopy
from ucl_masterclass import *
from keras.models import Sequential
from keras.layers import Dense

# This uses some preprocessing from sklearn you may need to add install sklearn (which is one of the other most common libraries)
# pip3 install -U scikit-learn
from sklearn.preprocessing import scale
import tensorflow as tf
from tensorflow import keras
from time import time

# Read in the train/test/validation datasets

In [ ]:

# Load the training, validation, and testing data
df_train = pd.read_csv(train_data_path)
df_val = pd.read_csv(val_data_path)
df_test = pd.read_csv(test_data_path)

### 3.2 Training Datasets 

The neural network is trained using a set of training events that have a label of 1 for signal and 0 for background. The dataset is split into *train*, *validation*, and *test*. Below, scaling is applied.

In [ ]:
# List of variables used in training
variables = ['dRBB','mBB','MET','Mtop','pTV',]

(x_train, y_train, w_train, 
 dset_val, dset_test) = scale_prepare_data(df_train, df_val, df_test, variables, scaler='standard')


### 3.3 Build a Simple Neural Network

Some simple code to train a very basic and sub-optimal NN, included to demonstrate how to evalute the performance and plot the output in Section 3.4.


In [ ]:
def classifier(variables):
    """
    Creates a model for higgs to bb classification
    
    returns: Keras model
    """
    num_variables = len(variables)

    model = Sequential()
    
    # The input layer
    model.add(Dense(units=num_variables,input_dim = num_variables,activation='relu'))
    
    # Add hidden layers here
    # ======================

    # Output layer
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='SGD', metrics=['accuracy'])
    return model

In [ ]:
start = time()

# Create and compile model

# Set these parameters
# ====================
epochs = 5
batchSize = 64
# train
model = classifier(variables)

history = model.fit(x_train, y_train, sample_weight = w_train,
        validation_data = dset_val, 
        epochs=epochs, batch_size=batchSize,verbose = 1)

print("model trained in " + str(round(time()-start,2))+"s")

### 3.4 Evaluating Neural Network

The code below will test a neural network on unseen events. In this case a neural network trained on the odd data set is tested on the even data set. The neural network is used to classify the events and calculate the sensitivity using the neural network output shown in the histogram below.



In [ ]:
df_test['decision_value'] = model.predict(dset_test[0])
print("A sensitivity of", round(sensitivity_NN(df_test)[0],2),'was achieved')
nn_output_plot(df_test)

**This material was produced by hackingEducation**  
<img src="../docs/images/logo-black.png" width="50" align = 'left'/>